In [1]:
pip install xgboost scikit-learn pandas joblib imbalanced-learn optuna

    stone (>=2.*)
           ~~~~^

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# Load Dataset
df = pd.read_csv("k8Dataset .csv")

# Drop unnecessary column (if present)
if "Slow Response Likelihood (%)" in df.columns:
    df = df.drop(columns=["Slow Response Likelihood (%)"])

# One-Hot Encode 'Event Type' (Categorical Column)
encoder = OneHotEncoder(sparse_output=False)
encoded_event_type = encoder.fit_transform(df[['Event Type']])
encoded_event_df = pd.DataFrame(encoded_event_type, columns=[f"Event_Type_{i}" for i in range(encoded_event_type.shape[1])])

# Merge Encoded Data and Drop Original Categorical Column
df = df.drop(columns=['Event Type']).reset_index(drop=True)
df = pd.concat([df, encoded_event_df], axis=1)

# Define Features (X) and Target Variables (Predict All 6 Parameters)
X = df.drop(columns=['Failure Type'])  # Features
y = df[['CPU Usage (%)', 'Memory Usage (%)', 'Disk Usage (%)', 'Network Latency (ms)', 'Packet Loss (%)', 'Pod Eviction Risk (%)']]  # Predict all 6

# Split Data into Training (80%) and Testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to Handle Class Imbalance
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Print Class Distribution After SMOTE
print("SMOTE applied - Training set balanced")


ValueError: Unknown label type: continuous-multioutput. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [3]:
# Define parameter grid for tuning
param_grid = {
    'n_estimators': [100, 200, 500],  # Number of trees
    'max_depth': [6, 8, 10],          # Depth of each tree
    'learning_rate': [0.01, 0.05, 0.1],  # Learning rate
    'gamma': [0, 0.1, 0.2],           # Regularization parameter
    'colsample_bytree': [0.6, 0.8, 1.0]  # Feature selection
}

# Initialize XGBoost model
xgb_model = xgb.XGBRegressor(eval_metric="rmse")  # Regression for continuous values

# Perform GridSearchCV for best parameters
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='r2', verbose=2, n_jobs=-1)
grid_search.fit(X_train_balanced, y_train_balanced)

# Print best parameters
print(f"Best Parameters: {grid_search.best_params_}")

# Train final model with best parameters
best_xgb_model = grid_search.best_estimator_


NameError: name 'X_train_balanced' is not defined

In [5]:
# Train XGBoost with best parameters
best_xgb_model.fit(X_train_balanced, y_train_balanced)

# Predict on Test Set
y_pred = best_xgb_model.predict(X_test)

# Evaluate Model Performance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Compute Error Metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R2 Score: {r2}")


NameError: name 'best_xgb_model' is not defined

In [6]:
# Train XGBoost with best parameters
best_xgb_model.fit(X_train_balanced, y_train_balanced)

# Predict on Test Set
y_pred = best_xgb_model.predict(X_test)

# Evaluate Model Performance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Compute Error Metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R2 Score: {r2}")


NameError: name 'best_xgb_model' is not defined

In [7]:
from sklearn.preprocessing import StandardScaler

# Define Features (X) and Target Variables (y)  
X = df.drop(columns=['Failure Type'])  # Features  
y = df[['CPU Usage (%)', 'Memory Usage (%)', 'Disk Usage (%)', 'Network Latency (ms)', 'Packet Loss (%)', 'Pod Eviction Risk (%)']]  # Target  

# Split Data into Training (80%) and Testing (20%)  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize Target Variables Using StandardScaler
scaler = StandardScaler()
y_train_scaled = scaler.fit_transform(y_train)
y_test_scaled = scaler.transform(y_test)

print("Data Preprocessing Complete (Features Split & Targets Scaled).")


Data Preprocessing Complete (Features Split & Targets Scaled).


In [8]:
import xgboost as xgb

# Train XGBoost for Multi-Target Regression
xgb_model = xgb.XGBRegressor(n_estimators=200, max_depth=8, learning_rate=0.05)

# Fit the model on training data
xgb_model.fit(X_train, y_train_scaled)

# Predict on Test Set
y_pred_scaled = xgb_model.predict(X_test)

# Convert Predictions Back to Original Scale
y_pred = scaler.inverse_transform(y_pred_scaled)

print("XGBoost Training Completed & Predictions Generated.")


XGBoost Training Completed & Predictions Generated.


In [9]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Compute Error Metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R2 Score: {r2}")


Mean Absolute Error (MAE): 0.0009077075178813525
Mean Squared Error (MSE): 6.32068463474448e-06
R2 Score: 0.9997857642117203


In [12]:
from sklearn.metrics import r2_score

# Compute overall model accuracy for predicting all 6 parameters
overall_r2_score = r2_score(y_test, y_pred)

print(f"Overall XGBoost Model Accuracy: {overall_r2_score * 100:.2f}%")


Overall XGBoost Model Accuracy: 99.98%


In [13]:
from sklearn.metrics import r2_score

# Make Predictions on Test Data
y_pred = best_xgb_model.predict(X_test)

# Compute Overall Accuracy
overall_r2_score = r2_score(y_test, y_pred)
print(f"✅ **Overall Model Accuracy on Test Data: {overall_r2_score * 100:.2f}%**")

# Show a Sample Test Prediction
sample_input = X_test.iloc[:1]  # Pick one sample row
sample_prediction = best_xgb_model.predict(sample_input)

print("\n📌 Sample Test Data Input:\n", sample_input)
print("\n🎯 Predicted Values:\n", sample_prediction)


NameError: name 'best_xgb_model' is not defined

In [14]:
import xgboost as xgb

# Define and Train XGBoost Model
best_xgb_model = xgb.XGBRegressor(n_estimators=200, max_depth=8, learning_rate=0.05)

best_xgb_model.fit(X_train, y_train)

print("✅ Model Trained Successfully!")


✅ Model Trained Successfully!


In [15]:
import joblib

# Save the trained model
joblib.dump(best_xgb_model, "xgboost_multitarget_model.pkl")

print("✅ Model saved successfully as 'xgboost_multitarget_model.pkl'")


✅ Model saved successfully as 'xgboost_multitarget_model.pkl'


In [16]:
# Load the saved model
loaded_model = joblib.load("xgboost_multitarget_model.pkl")

# Test Predictions
y_pred = loaded_model.predict(X_test)

# Compute Accuracy
from sklearn.metrics import r2_score
overall_r2_score = r2_score(y_test, y_pred)

print(f"🎯 Model Accuracy on Test Data: {overall_r2_score * 100:.2f}%")


🎯 Model Accuracy on Test Data: 99.98%


In [17]:
from sklearn.metrics import r2_score

# Load the saved model
import joblib
loaded_model = joblib.load("xgboost_multitarget_model.pkl")

# Make Predictions on Test Data
y_pred = loaded_model.predict(X_test)

# Compute Overall Accuracy
overall_r2_score = r2_score(y_test, y_pred)
print(f"✅ **Overall Model Accuracy on Test Data: {overall_r2_score * 100:.2f}%**")

# Show a Sample Test Prediction
sample_input = X_test.iloc[:1]  # Pick one sample row
sample_prediction = loaded_model.predict(sample_input)

print("\n📌 Sample Test Data Input:\n", sample_input)
print("\n🎯 Predicted Values:\n", sample_prediction)


✅ **Overall Model Accuracy on Test Data: 99.98%**

📌 Sample Test Data Input:
       CPU Usage (%)  Memory Usage (%)  Disk Usage (%)  Network Latency (ms)  \
6252            1.0          0.561822        0.663669              0.285373   

      Packet Loss (%)  Error Logs Count  Pod Eviction Risk (%)  Event_Type_0  \
6252          0.51974               0.2                    0.0           0.0   

      Event_Type_1  Event_Type_2  
6252           1.0           0.0  

🎯 Predicted Values:
 [[9.8441589e-01 5.6121349e-01 6.6250628e-01 2.8536391e-01 5.1865375e-01
  1.4142146e-05]]


In [18]:
pip install flask


Note: you may need to restart the kernel to use updated packages.


    stone (>=2.*)
           ~~~~^

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd

# Load the Trained Model
model = joblib.load("xgboost_multitarget_model.pkl")

# Initialize Flask App
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    # Get JSON data from request
    data = request.get_json()

    # Convert JSON to DataFrame
    input_data = pd.DataFrame([data])

    # Make Prediction
    prediction = model.predict(input_data)

    # Convert prediction to JSON format
    response = {
        "CPU Usage (%)": float(prediction[0][0]),
        "Memory Usage (%)": float(prediction[0][1]),
        "Disk Usage (%)": float(prediction[0][2]),
        "Network Latency (ms)": float(prediction[0][3]),
        "Packet Loss (%)": float(prediction[0][4]),
        "Pod Eviction Risk (%)": float(prediction[0][5]),
    }

    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\User\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [21]:
import pandas as pd

# Select 5 random rows from the test set
sample_test_data = X_test.sample(5, random_state=42)

# Print sample test data
print("\n📌 Sample Testing Data (Input):")
print(sample_test_data)



📌 Sample Testing Data (Input):
      CPU Usage (%)  Memory Usage (%)  Disk Usage (%)  Network Latency (ms)  \
4343       0.705022          0.572458        0.465281              0.789997   
1794       0.542639          0.830732        0.233664              0.019865   
708        0.083614          0.454354        0.292465              0.231112   
3383       0.705397          0.732097        0.183200              0.143427   
4534       0.648492          0.442284        0.505883              0.631654   

      Packet Loss (%)  Error Logs Count  Pod Eviction Risk (%)  Event_Type_0  \
4343         0.380551               0.1               0.000000           0.0   
1794         0.432600               0.2               0.843415           0.0   
708          0.304388               0.2               0.000000           0.0   
3383         0.347588               0.3               0.000000           0.0   
4534         0.514232               0.2               0.000000           0.0   

      Event_

In [22]:
import joblib

# Load the saved XGBoost model
model = joblib.load("xgboost_multitarget_model.pkl")

# Predict on sample test data
sample_predictions = model.predict(sample_test_data)

# Print Predictions
print("\n🎯 Predictions for Sample Testing Data:")
for i, row in enumerate(sample_predictions):
    print(f"\n🔹 Test Sample {i+1}:")
    print(f"   - CPU Usage (%): {row[0]:.2f}")
    print(f"   - Memory Usage (%): {row[1]:.2f}")
    print(f"   - Disk Usage (%): {row[2]:.2f}")
    print(f"   - Network Latency (ms): {row[3]:.2f}")
    print(f"   - Packet Loss (%): {row[4]:.2f}")
    print(f"   - Pod Eviction Risk (%): {row[5]:.2f}")



🎯 Predictions for Sample Testing Data:

🔹 Test Sample 1:
   - CPU Usage (%): 0.71
   - Memory Usage (%): 0.57
   - Disk Usage (%): 0.47
   - Network Latency (ms): 0.79
   - Packet Loss (%): 0.38
   - Pod Eviction Risk (%): 0.00

🔹 Test Sample 2:
   - CPU Usage (%): 0.54
   - Memory Usage (%): 0.83
   - Disk Usage (%): 0.23
   - Network Latency (ms): 0.02
   - Packet Loss (%): 0.43
   - Pod Eviction Risk (%): 0.84

🔹 Test Sample 3:
   - CPU Usage (%): 0.09
   - Memory Usage (%): 0.45
   - Disk Usage (%): 0.29
   - Network Latency (ms): 0.23
   - Packet Loss (%): 0.30
   - Pod Eviction Risk (%): 0.00

🔹 Test Sample 4:
   - CPU Usage (%): 0.71
   - Memory Usage (%): 0.73
   - Disk Usage (%): 0.18
   - Network Latency (ms): 0.14
   - Packet Loss (%): 0.35
   - Pod Eviction Risk (%): 0.00

🔹 Test Sample 5:
   - CPU Usage (%): 0.65
   - Memory Usage (%): 0.44
   - Disk Usage (%): 0.51
   - Network Latency (ms): 0.63
   - Packet Loss (%): 0.51
   - Pod Eviction Risk (%): 0.00


In [23]:
from sklearn.metrics import r2_score

# Predict on full test set
y_pred = model.predict(X_test)

# Compute Overall Accuracy
overall_r2_score = r2_score(y_test, y_pred)
print(f"\n✅ **Overall Model Accuracy on Test Data: {overall_r2_score * 100:.2f}%**")



✅ **Overall Model Accuracy on Test Data: 99.98%**


In [24]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Make Predictions on Test Data
y_pred = model.predict(X_test)

# Convert predictions & actual values to class labels (if needed)
y_test_labels = np.argmax(y_test, axis=1)
y_pred_labels = np.argmax(y_pred, axis=1)

# Compute Accuracy
overall_accuracy = accuracy_score(y_test_labels, y_pred_labels) * 100

# Generate Classification Report
report = classification_report(y_test_labels, y_pred_labels, digits=2)

# Print Model Accuracy & Report in Desired Format
print(f"XGBoost Model Accuracy: {overall_accuracy:.2f}%\n")
print(report)


XGBoost Model Accuracy: 99.70%

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       606
           1       0.99      1.00      1.00       666
           2       1.00      0.99      1.00       250
           3       0.99      1.00      0.99       161
           4       1.00      0.99      1.00       266
           5       1.00      0.98      0.99        51

    accuracy                           1.00      2000
   macro avg       1.00      0.99      1.00      2000
weighted avg       1.00      1.00      1.00      2000

